# Objective
Predict resale prices of BMW cars. This could for instance be used by someone who wants to sell their car, to get an idea about how much it is worth, similar to how Kelley Blue Book works.

# Thinking about the problem
From the readme of the dataset available here <https://github.com/datacamp/careerhub-data/tree/master/BMW%20Used%20Car%20Sales>, one can see that the dataset contains information about price, transmission, mileage, fuel type, road tax, miles per gallon (mpg), and engine size. Upon inspection of the dataset (see below), it turned out to additionally contain the car model and year (I'm assuming this means production year). First I want to describe my initial expectations for the relationships between these quantities, and formulate different levels of complexity for including the data.

The five quantities model, year, transmission, fuel type, and engine size collectively describe the car configuration at the time of initial purchase. The quantity mileage describes how much the car has been used, and therefore worn since that point. The quantities miles per gallon and road tax should be given based on the new car configuration quantities.

I suspect that the price will strongly depend on the mileage and age of the car, and a first simple model could therefore just consider these two variables.

In [ ]:
# This requires the file draw_diagrams.py to be in the same directory as this notebook
import draw_diagrams
draw_diagrams.data_model1()

An improvement on this would be to include the new car configuration variables. From these in addition to price, mpg and road tax could be inferred.

In [ ]:
draw_diagrams.data_model2()

Finally the last two variables, mpg and road tax, can be included. These could affect the resale price of the car, since they would probably influence how much a buyer is willing to pay, but I suspect this connection will be less strong than the connection between the other variables and price.

In [ ]:
draw_diagrams.data_model3()

Before any of this though, first I want to take a closer at the data.


# Loading and inspecting data
First I load and inspect the data. I downloaded the data from [here](https://raw.githubusercontent.com/datacamp/careerhub-data/master/BMW%20Used%20Car%20Sales/bmw.csv) and saved it in the `datasets/bmw.csv` file.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
bmw = pd.read_csv("datasets/bmw.csv")
bmw.head()

In [ ]:
bmw.info()

In [ ]:
def print_category_values(bmw):
    for col in ["model", "fuelType", "transmission"]:
        print(col)
        print(list(bmw[col].unique()))


print_category_values(bmw)

In [ ]:
bmw.describe()

In [ ]:
for col in bmw:
    print(col, len(bmw[col].unique()))

# Data exploration
<a id = "data-exploration"></a>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Let's look at how the price depends on all the continous variables using a pair plot

In [ ]:
sns.pairplot(
    bmw,  # hue='transmission',
    x_vars=["price", "year", "mileage", "tax", "mpg", "engineSize"],
    y_vars=["price"],
)

There appears to be a definite relationship between price and both mileage and year. The relationship looks like at might be expopnential, so let's we look at the logarithm of the price

In [ ]:
bmw_log = bmw.copy()
bmw_log['log price'] = np.log10(bmw_log['price'])
bmw_log = bmw_log.drop('price', axis='columns')

sns.pairplot(bmw_log, #hue='transmission', 
             x_vars=['log price', 'year', 'mileage',  'tax', 'mpg', 'engineSize'],
             y_vars=['log price']) #, hue='transmission')

These plots reveal that there appears to be a linear relationship between the logarithm of the price, and both year and mileage. There is no obvious relationship between the price and the remaining variables, whether we consider logarithm or not. Going forward in the analysis, we will be using the logarithm of the price as the target variable.

# Data cleaning
## Categorical variables
Let us take a closer look at the categorical columns. First we print the number of values in each category

In [ ]:
categorical_columns = ["model", "fuelType", "transmission"]


def print_categorical_counts(df, columns):
    for col in columns:
        display(df.groupby(col)[col].count())


print_categorical_counts(bmw_log, categorical_columns)

There are a number of categories with very few records. For instance, the `fuelType` `Electric` has only three. With such a small amount of observations for this category, and no obvious relationship with other entries in this category as one naturally has for numeric columns, I wouldn't expect it to be possible to make reliable predictions for the selling price for this category. I therefore choose to drop any category with less than 10 records. 

In [ ]:
def drop_almost_empty_categories(df, col, nmin=10):
    df = df.copy()  # To avoid modyfiyng the input dataframe
    category_count = df.groupby(col)[col].count()
    for category_name, count in category_count.iteritems():
        if count < nmin:
            df = df[df[col] != category_name]
    return df


bmw_cat = bmw_log.copy()
for col in categorical_columns:
    bmw_cat = drop_almost_empty_categories(bmw_cat, col)
bmw_cat[categorical_columns] = bmw_cat[categorical_columns].astype('category')
# print_categorical_counts(bmw_cat, categorical_columns)

In [ ]:
new_car_config_cols = ['model', 'transmission', 'fuelType', 'engineSize']
new_car_cols = new_car_config_cols + ['year']

In [ ]:
sns.pairplot(
    bmw.sort_values("engineSize"),  # hue='transmission',
    x_vars=new_car_cols,
    y_vars=new_car_cols,
)

In [ ]:
bmw_cat[bmw_cat.engineSize==0].head()

In [ ]:
with pd.option_context("display.max_rows", None):
    new_car_grouped = bmw.groupby(new_car_cols)[["tax", "mpg", "price"]]
    display(new_car_grouped.nunique())
    # display(bmw.groupby(new_car_config_cols)['tax'].nunique())

In [ ]:
choices = (
    (bmw.model == " 1 Series")
    & (bmw.transmission == "Automatic")
    & (bmw.fuelType == "Diesel")
    & (bmw.engineSize == 2.0)
    & (bmw.year == 2016)
)
bmw[choices][["mileage", "tax", "mpg", "price"]].sort_values("mileage")

## A bit more data cleaning

From the plots we can see that `mpg` has a group of values near 400, far from the nearest values which are less than 200. Let's see how many different values  are present there

In [ ]:
bmw_cat[bmw_cat["mpg"]>400]["mpg"].unique()

All the values of `mpg` in the group near 400 have the same value. This looks very suspicious. I suspect this is data is wrong, and since it could seriously skew a model since it has such high values, I should eliminate these values (either impute with e.g. average, or drop the records all together).

Let's also check the remaining two continous variables

In [ ]:
# display(sorted(bmw_dropped["engineSize"].unique()))
display(bmw_cat.groupby("engineSize")["engineSize"].count())
bmw_cat.groupby("tax")["tax"].count()

They both contain zeros, which seems weird for both tax and engine size. The skewing effect is probably less then for the `mpg` outliers, since zero is closer to other values of tax and engine size, but I should still either impute or drop these records.

In [ ]:
to_be_dropped = (bmw_cat.mpg > 400) | (bmw_cat.engineSize == 0) | (bmw_cat.tax == 0)
bmw_cleaned = bmw_cat[~to_be_dropped]
# bmw_cleaned = bmw_log
# bmw_cleaned.info()

# Regression models
Here I train models on the data.

## Linear regression models
For the first model, I only want to consider the dependency of price on build year and mileage. From the plots in the [data exploration](#data-exploration) section we see that the logarithm of the price appears to depend linearly on year and mileage.

In [ ]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


def print_linear_coeffs(features, linreg, std=None):
    coeffs = pd.DataFrame(
        {
            "observable": features,
            "coef": linreg.coef_,
            "10^coef": np.power(10, linreg.coef_),
        }
    )
    coeffs = coeffs.set_index("observable")
    if std is not None:
        coeffs["std"] = std
        coeffs["coef*std"] = coeffs["std"] * coeffs["coef"]
        coeffs = coeffs.sort_values("coef*std", key=np.abs, ascending=False)
    display(coeffs)


def every_column_name_but(df, dependent):
    features = [col for col in df.columns if col != dependent]
    return features


def split_dependent(df, features="all", dependent="log price"):
    if features == "all":
        features = every_column_name_but(df, dependent)
    else:
        features = [col for col in features if col in df.columns and col != dependent]
    return df[features], df[dependent]

In [ ]:
bmw_train, bmw_val = train_test_split(bmw_cleaned, test_size=0.2, random_state=42)

In [ ]:
def make_cat_ohe(drop="first"):
    """Make a one hot encoder that only acts on categorical columns"""
    cat_transformer_tuple = (
        OneHotEncoder(drop=drop),
        make_column_selector(dtype_include="category"),
    )
    ohe = make_column_transformer(cat_transformer_tuple, remainder="passthrough")
    return ohe

ohe = make_cat_ohe()

linreg = Pipeline((("one_hot", ohe), ("regressor", LinearRegression())))

In [ ]:
X, y = split_dependent(bmw_train[["log price", "mileage", "year"]], dependent="log price")
cross_validate(linreg, X, y, return_train_score=True)

In [ ]:
def scores_mean_and_std(scores):
    """Finds mean and standard deviations of scores from `cross_validate`,
    and puts them in a dataframe."""
    scores = pd.DataFrame(scores)[["test_score", "train_score"]]
    mean = scores.mean().add_prefix("mean_")
    std = scores.std().add_prefix("std_")
    mean_std = pd.concat((mean, std))
    return mean_std

feature_cols = ["mileage", "model", "year", "engineSize", "transmission", "fuelType", "mpg", "tax"]
#feature_cols = ["mileage", "model", "year", "engineSize", "fuelType", "transmission", "mpg", "tax"]
linreg = Pipeline((("one_hot", make_cat_ohe()), ("regressor", LinearRegression())))
all_scores = {}
for i in range(1, len(feature_cols) + 1):
    cols = ["log price"] + feature_cols[:i]
    X, y = split_dependent(bmw_train[cols], dependent="log price")
    scores = cross_validate(linreg, X, y, return_train_score=True)
    all_scores[cols[-1]] = scores_mean_and_std(scores)
all_scores = pd.DataFrame(all_scores).T
all_scores.index.name = "Last added feature"
display(all_scores)

Here I cumulatively added features one by one, and look at the five-fold cross validation score from fitting a linear model. I see that only considering the `mileage` gives a low R^2 score of 0.432. Adding the car `model` improves it considerably, as does adding `year`. Further adding the remaining new car configuration features further improves the R^2 score. Adding the `mpg` and `tax` does not change the R^2 score much though. We therefore continue the analysis including only the `mileage` and the new car configuration features, but excluding `mpg` and `tax`.

In [ ]:
cols = [
    "log price",
    "mileage",
    "model",
    "year",
    "engineSize",
    "transmission",
    "fuelType",
]
bmw_reduced = bmw_cleaned[cols]
bmw_reduced_train = bmw_train[cols]
bmw_reduced_val = bmw_val[cols]

## Lasso and Ridge
The training and test scores of the linear models fitted are not very different, which indicates that the model is not overfitting much. This is also expectable for linear models, since these models tend to have large bias, but low variance.

To make sure that the data has not been overfitted, I performed lasso and ridge regressions, for a series of values of their `alpha` parameters.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge

lasso = Pipeline((("one_hot", make_cat_ohe()), ("regressor", Lasso())))

param_grid = {"regressor__alpha": [0.0005, 0.001, 0.01, 0.1]}
clf = GridSearchCV(estimator=lasso, param_grid=param_grid, return_train_score=True)
X, y = split_dependent(bmw_reduced, dependent="log price")
clf.fit(X, y)
display(
    pd.DataFrame(clf.cv_results_)[
        [
            "param_regressor__alpha",
            "mean_test_score",
            "mean_train_score",
            "std_test_score",
            "std_train_score",
        ]
    ].set_index("param_regressor__alpha")
)

In [ ]:

ridge = Pipeline((("one_hot", make_cat_ohe()), ("regressor", Ridge(tol=1e-9))))

param_grid = {"regressor__alpha": [0] + list(10**i for i in range(5))}
clf = GridSearchCV(estimator=ridge, param_grid=param_grid, return_train_score=True)
clf.fit(X, y)
display(
    pd.DataFrame(clf.cv_results_)[
        [
            "param_regressor__alpha",
            "mean_test_score",
            "mean_train_score",
            "std_test_score",
            "std_train_score",
        ]
    ].set_index("param_regressor__alpha")
)


We see that neither lasso or ridge regression are able to decrease the difference between test and the training score, which I also expected from the small initial difference between the two. I therefore continue the analysis without any of the regularizers.

## Validation check
Finally, I check the model using the validation set.

In [ ]:
X, y = split_dependent(bmw_reduced_train, dependent="log price")
linreg.fit(X, y)
X_val, y_val = split_dependent(bmw_reduced_val, dependent="log price")
y_predict_val = linreg.predict(X_val)
r2_score(y_val, y_predict_val)

The obtained validation R^2 score is close to the R^2 from the cross-validation, which once again indicates that the model is not overfitting. I go ahead with this model, refitting it to the full dataset.

In [ ]:
X, y = split_dependent(bmw_reduced, dependent="log price")
linreg.fit(X, y)

## Parameter interpretation
A nice property of the linear regression model is that it's coefficients has straightforward interpretations. Below I print these coefficients, together with the standard deviation in the corresponding variable.

In [ ]:
cat_cols = bmw_reduced.columns[bmw_reduced.dtypes == "category"]


def rename_ohe_features(features, cat_cols):
    for i, cat_col in enumerate(cat_cols):
        features = [
            feature.replace(f"onehotencoder__x{i}", cat_col) for feature in features
        ]
    return features


features = linreg.named_steps["one_hot"].get_feature_names()
features = rename_ohe_features(features, cat_cols)

std = pd.get_dummies(bmw_reduced).std()
print_linear_coeffs(features, linreg.named_steps["regressor"], std)

I sorted the coefficients by the product of the coefficient and the standard deviation. This product gives a measure of how important the feature is in the model. Since we fit to the logarithm of the price, I also show $10^{\text{coeff}}$. This can be interpreted as multiplicative factor, modifying the price depending on the value of the feature. For instance, `year` has a value of $10^\text{coef}=1.11$, which means that a car would be 1.107 times more expensive than a corresponding one year older car.
For the categorical variables, such as model, $10^\text{coef}$ describes the relative price between the different categories. For instance, `model_X5` has $10^\text{coef}=1.71$, meaning that an X5 car is 1.71 times more expensive than the first model, which was dropped by the `OneHotEncoder`. How many times more expensive one model is compared to another can be found by dividing their values of $10^\text{coef}$.

## Prediction intervals
Only a single number of returned by the linear model above when it is given the data for a car. But we would generally expect that the real prices are distributed with some variance around this value. It would be nice though to have some kind of idea as to how accurate that number is. One way to indicate this is with a prediction interval.
A prediction interval is an interval of prices, in which we with some percentage (say 90%) of confidence can say that the price of the car would be within. Note that this is different from the confidence interval, which specifies how confidently we can say that we have predicted the mean distribution, but it says nothing about the variance.

Since prediction intervals does not appear to be included in scikit-learn, I will do my own simple implementation here.
From <https://online.stat.psu.edu/stat501/lesson/3/3.3> we have the following expression for the prediction interval

$$\hat{y}_h \pm t_{(1-\alpha/2, n-2)}\sqrt{MSE \cdot \left(1+\frac{1}{n}+\frac{(x_h-\bar{x})^2}{\sum_i(x_i-\bar{x})^2}\right)}$$

where $\hat{y}_k$ is the fitted value, $t_{(1-\alpha/2, n-2)}$ is the pdf of a t-distribution and $n$ is the number of samples.

In [ ]:
from scipy.stats import t
n = len(bmw_reduced)
alpha=0.90
t.ppf((1+alpha)/2, n-2)
MSE = mean_squared_error(y, linreg.predict(X))
X_ohe = linreg.named_steps["one_hot"].transform(X)
X_ohe - np.mean(X_ohe, axis=0)

Following <https://saattrupdan.github.io/2020-02-26-parametric-prediction/> I implemented

In [ ]:
from scipy.stats import t
def calc_prediction_delta(y, y_pred, alpha=0.90, print_ratio_captured=False):
    n = len(y)
    resid = y-y_pred
    mean_resid = np.mean(y-y_pred)
    sN2 = 1/(n-1)*sum((resid-mean_resid)**2)
    dy = t.ppf((1+alpha)/2, n-1)*np.sqrt(sN2)*(1+1/n)
    if print_ratio_captured:
        print("Ratio in prediction interval", np.mean(np.abs(resid + mean_resid) < dy))
    return dy

In [ ]:
for i in range(1, 12):
    X_, y_ = split_dependent(bmw_reduced[:i*1000])
    dy = calc_prediction_delta(y_, linreg.predict(X_), print_ratio_captured=True)

Calculating this over the BMW dataset gives

In [ ]:
dy = calc_prediction_delta(y, linreg.predict(X), alpha=0.90, print_ratio_captured=True)
print("dy", dy)
print("10^dy", 10**dy)

Here we see that indeed around 90% of the prices were within the 90% prediction interval.

In [ ]:
def price_with_pred_interval(X, linreg, dy):
    y_predict = linreg.predict(X[:10])
    y_pred_w_interval = pd.DataFrame(
        {"y": y_predict, "y-dy": y_predict - dy, "y+dy": y_predict + dy}
    )
    price = np.power(10, y_pred_w_interval).rename(
        {"y": "price", "y-dy": "upper", "y+dy": "lower"}, axis="columns"
    )
    print(price)


price_with_pred_interval(X[:10], linreg, dy)

In [ ]:
feature_cols = ["mileage", "model", "year", "engineSize", "transmission", "fuelType", "mpg", "tax"]
#feature_cols = ["mileage", "model", "year", "engineSize", "fuelType", "transmission", "mpg", "tax"]
linreg_play = Pipeline((("one_hot", make_cat_ohe()), ("regressor", LinearRegression())))
all_dys = {}
for i in range(1, len(feature_cols) + 1):
    cols = ["log price"] + feature_cols[:i]
    X, y = split_dependent(bmw_train[cols], dependent="log price")
    linreg_play.fit(X, y)
    dy = calc_prediction_delta(y, linreg_play.predict(X), alpha=0.90, print_ratio_captured=True)
    all_dys[cols[-1]] = dy
all_dys = pd.Series(all_dys)
all_dys = pd.DataFrame({"dy": all_dys, "10^dy": np.power(10, all_dys)})

all_dys.index.name = "Last added feature"
display(all_dys)

# Making predictions with partial data

In [ ]:
from itertools import chain, combinations
def powerset(iterable, start=0):
    """"powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    This function comes from the python documentation at https://docs.python.org/3/library/itertools.html"""
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(start, len(s)+1))

def train_set_of_models(features, bmw, dependent="log price"):
    models = {}
    for feature_set in powerset(features, start=1):
        linreg_local = Pipeline((("one_hot", make_cat_ohe()), ("regressor", LinearRegression())))
        X = bmw[list(feature_set)]
        y = bmw[dependent]
        linreg_local.fit(X, y)
        dy = calc_prediction_delta(y, linreg_local.predict(X), alpha=0.90)
        models[feature_set] = {"model": linreg_local, "dy_90": dy}
    return models
#        print("feature_set", feature_set)


included_features = ["mileage", "model", "year", "engineSize", "transmission"]
#included_features = included_features[:2]
models = train_set_of_models(included_features, bmw_cleaned)



In [ ]:
def find_longest_element(keys):
    return keys[np.argmax(list(map(len, keys)))]


def scalar_to_list(x):
    if np.isscalar(x):
        return [x]
    else:
        return x


def eval_model(models, **kwargs):
    features = find_longest_element(list(models.keys()))
    #print(features)
    for key in kwargs:
        if key not in features:
            raise ValueError(f"{key} not found in {features}")
    chosen_features = tuple(feature for feature in features if feature in kwargs)
    model_holder = models[chosen_features]
    model = model_holder["model"]
    dy = model_holder["dy_90"]

    values = (scalar_to_list(kwargs[feature]) for feature in chosen_features)
    X = pd.DataFrame(
        dict(
            zip(
                chosen_features,
                values,
            )
        )
    )
    price = np.power(10, model.predict(X))
    lower_90 = np.power(10, model.predict(X)-dy)
    upper_90 = np.power(10, model.predict(X)+dy)
    price_w_interval = pd.DataFrame(
        {"price": price, "90% lower bound": lower_90, "90% upper bound": upper_90}
    )

    return price_w_interval


eval_model(models, mileage=[20, 400, 500], model=3*[" 2 Series"])
#eval_model(models, mileage=[20], model=" 2 Series")

In [ ]:
print_category_values(bmw)

# Dumping models

In [ ]:
from pickle import dump
model_dump_file = "bmw_linreg_model.pckl"
with open(model_dump_file, "wb") as file_:
    dump(models, file_)

In [ ]:
import json
def extract_feature_ranges(df):
    features_ranges = {}
    for col in df.select_dtypes(include=np.number):
        series = df[col]
        summary = {
            "type": "numeric",
            "range": (float(series.min()), float(series.max())),
        }
        features_ranges[col] = summary
    for col in df.select_dtypes(include="category"):
        series = df[col]
        summary = {"type": "category", "values": list(series.cat.categories)}
        features_ranges[col] = summary
    return features_ranges


def dump_feature_ranges_to_json_file(df, filename="feature_ranges.json"):
    feature_ranges = extract_feature_ranges(df)
    print(feature_ranges)
    with open(filename, "w") as fil:
        json.dump(feature_ranges, fil)


dump_feature_ranges_to_json_file(X[included_features])

In [ ]:
for key, model_dict in models.items():
    print(round(100*model_dict["dy_90"]), key, )